# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd

!pip install langdetect

from langdetect import detect

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

    100% |████████████████████████████████| 1.0MB 16.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for langdetect ... done
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql_table('Table', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.shape

(26216, 40)

In [5]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [6]:
X = df['message']
Y = df.iloc[:,4:41]

In [7]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [8]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
'''
Test to detect language, actually works but not part of the project, as can only input message in english
def get_lang(text):
    try:
        lang = detect(text)
    except:
        lang = None
        
    return lang

X['language'] = df['original'].apply(lambda x: get_lang(x))
'''

"\nTest to detect language, actually works but not part of the project, as can only input message in english\ndef get_lang(text):\n    try:\n        lang = detect(text)\n    except:\n        lang = None\n        \n    return lang\n\nX['language'] = df['original'].apply(lambda x: get_lang(x))\n"

In [10]:
## X['language'].head()

In [11]:
## X['language'].unique()

In [12]:
## print(len(X['language'].unique()))

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
## I'll take more than the message (but problem later on when using the model only on message)
## Need to compare accuracies


## Readability of message, complexity (syllables) https://www.geeksforgeeks.org/readability-index-pythonnlp/
## Multioutput classifier https://stackoverflow.com/questions/51802752/sklearn-how-to-use-multioutputclassifier-with-multi-label-text-classification
## https://scikit-learn.org/stable/modules/multiclass.html
rf = RandomForestClassifier()
gbc = GradientBoostingClassifier()
ada = AdaBoostClassifier()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(ada))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)


In [16]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline.predict(X_test)

In [18]:
# Transform y_pred into DataFrame
y_pred_df = pd.DataFrame(y_pred, columns = Y.columns)
y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
for column in y_test.columns:
    print('Model Performance with Category: {}'.format(column))
    print(classification_report(y_test[column],y_pred_df[column]))

Model Performance with Category: related
             precision    recall  f1-score   support

          0       0.69      0.10      0.17      1563
          1       0.77      0.99      0.86      4944
          2       0.36      0.09      0.14        47

avg / total       0.75      0.77      0.69      6554

Model Performance with Category: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.77      0.48      0.59      1111

avg / total       0.88      0.89      0.88      6554

Model Performance with Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       0.17      0.03      0.05        33

avg / total       0.99      0.99      0.99      6554

Model Performance with Category: aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3884
          1       0.75      0

In [20]:
(y_pred == y_test).mean().mean()

0.94689841657342433

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
parameters = {'clf__estimator__n_estimators' : [1,10,30,50],
             'clf__estimator__base_estimator' :[rf, gbc]}

cv = None
model = GridSearchCV(pipeline, param_grid = parameters, cv = cv)

In [22]:
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print(model.best_params_)

In [ ]:
pipeline.get_params().keys()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred2 = model.predict(X_test)
print((y_pred2 == y_test).mean().mean())

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
rbf_svc = svm.SVC(kernel='rbf')

ada = AdaBoostClassifier()
rf = RandomForestClassifier()


In [24]:
# create custom transformer

from sklearn.base import BaseEstimator, TransformerMixin

class TextLengthExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(pd.Series(X).apply(lambda x: len(x)))

In [25]:
# new tokenize function



def tokenize2(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [27]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize2(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [28]:
new_model = Pipeline([
    ('features', FeatureUnion([
        
        ('nlp_pipeline', Pipeline([
             ('vect', CountVectorizer(tokenizer = tokenize)),
             ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor()),
        
        ('text_length', TextLengthExtractor())
    ])),
    
   
    ('clf', MultiOutputClassifier(ada))
])

In [29]:
new_model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [31]:
parameters = {
        'features__nlp_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__nlp_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__nlp_pipeline__vect__max_features': (None, 5000, 10000),
        'features__nlp_pipeline__tfidf__use_idf': (True, False),
    }

cv = GridSearchCV(new_model, param_grid=parameters)

In [32]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'features__nlp_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__nlp_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__nlp_pipeline__vect__max_features': (None, 5000, 10000), 'features__nlp_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [33]:
print(cv.best_params_)

{'features__nlp_pipeline__tfidf__use_idf': True, 'features__nlp_pipeline__vect__max_df': 0.5, 'features__nlp_pipeline__vect__max_features': 10000, 'features__nlp_pipeline__vect__ngram_range': (1, 1)}


In [34]:
new_y_pred = cv.predict(X_test)

In [35]:
(new_y_pred == y_test).mean().mean()

0.94700013562540286

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.